* one convolution notebook

In [3]:
# tensorflow on 8889

In [5]:
import os
os.chdir("/notebooks/src")

In [1]:
pwd

u'/notebooks/src'

In [4]:
! ls /home

In [7]:
!pip install keras cython h5py

    100% |################################| 276kB 2.2MB/s ta 0:00:01
    100% |################################| 2.9MB 450kB/s eta 0:00:01
    100% |################################| 256kB 5.1MB/s eta 0:00:01
  Running setup.py bdist_wheel for pyyaml ... done
  Stored in directory: /root/.cache/pip/wheels/2c/f7/79/13f3a12cd723892437c0cfbde1230ab4d82947ff7b3839a4fc
Successfully built pyyaml


In [39]:
from keras.layers import Input,Convolution2D, Dense, Activation, Flatten, merge, Conv2D
from keras.layers import MaxPooling2D, Dropout, LocallyConnected2D
from keras.models import Model, load_model
from keras.optimizers import SGD
import numpy as np
import pandas as pd
#from lib.data_split import load_train_data, load_test_data

* preprocessing the data

In [2]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
'''
This module takes a file path and loades pre_extracted features 
from leaf kaggle competiton. The input is the file path for
train or test files. 
'''

#train_df = pd.read_csv("./data/train.csv")
#id_1 = train_df.pop("id")
#target = train_df.pop("species")


def load_train(filepath):

    #loading
    train_df = pd.read_csv(filepath)
    id = train_df.pop('id')
    target = train_df.pop('species')
    
    #scaling
    target = LabelEncoder().fit(target).transform(target)
    features = StandardScaler().fit(train_df).transform(train_df)
    
    return id,features,target
    

def load_test(filepath):
    
    #loading
    test_df = pd.read_csv(filepath)
    id = test_df.pop('id')
    
    #scaling
    features = StandardScaler().fit(test_df).transform(test_df)
    
    return id, features
    

In [38]:
#train_df.head()

,margin1,margin2,margin3,margin4,margin5,margin6,margin7,margin8,margin9,margin10,...,texture55,texture56,texture57,texture58,texture59,texture60,texture61,texture62,texture63,texture64
0,0.007812,0.023438,0.023438,0.003906,0.011719,0.009766,0.027344,0.0,0.001953,0.033203,...,0.007812,0.000000,0.002930,0.002930,0.035156,0.0,0.0,0.004883,0.000000,0.025391
1,0.005859,0.000000,0.031250,0.015625,0.025391,0.001953,0.019531,0.0,0.000000,0.007812,...,0.000977,0.000000,0.000000,0.000977,0.023438,0.0,0.0,0.000977,0.039062,0.022461
2,0.005859,0.009766,0.019531,0.007812,0.003906,0.005859,0.068359,0.0,0.000000,0.044922,...,0.154300,0.000000,0.005859,0.000977,0.007812,0.0,0.0,0.000000,0.020508,0.002930
3,0.000000,0.003906,0.023438,0.005859,0.021484,0.019531,0.023438,0.0,0.013672,0.017578,...,0.000000,0.000977,0.000000,0.000000,0.020508,0.0,0.0,0.017578,0.000000,0.047852
4,0.005859,0.003906,0.048828,0.009766,0.013672,0.015625,0.005859,0.0,0.000000,0.005859,...,0.096680,0.000000,0.021484,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.031250


In [39]:
#test_df.head()

,margin1,margin2,margin3,margin4,margin5,margin6,margin7,margin8,margin9,margin10,...,texture55,texture56,texture57,texture58,texture59,texture60,texture61,texture62,texture63,texture64
0,0.019531,0.009766,0.078125,0.011719,0.003906,0.015625,0.005859,0.0,0.005859,0.023438,...,0.006836,0.000000,0.015625,0.000977,0.015625,0.0,0.0,0.000000,0.003906,0.053711
1,0.007812,0.005859,0.064453,0.009766,0.003906,0.013672,0.007812,0.0,0.033203,0.023438,...,0.000000,0.000000,0.006836,0.001953,0.013672,0.0,0.0,0.000977,0.037109,0.044922
2,0.000000,0.000000,0.001953,0.021484,0.041016,0.000000,0.023438,0.0,0.011719,0.005859,...,0.128910,0.000000,0.000977,0.000000,0.000000,0.0,0.0,0.015625,0.000000,0.000000
3,0.000000,0.000000,0.009766,0.011719,0.017578,0.000000,0.003906,0.0,0.003906,0.001953,...,0.012695,0.015625,0.002930,0.036133,0.013672,0.0,0.0,0.089844,0.000000,0.008789
4,0.001953,0.000000,0.015625,0.009766,0.039062,0.000000,0.009766,0.0,0.005859,0.000000,...,0.000000,0.042969,0.016602,0.010742,0.041016,0.0,0.0,0.007812,0.009766,0.007812


* loading images

In [10]:
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import img_to_array, load_img

'''
This module loads all the training/test images, resizes them and 
makes them ready for kaggle.
'''


def rebuild_image(img, dim=64):
    
    # find largest axis
    maxsize = max((0,1), key=lambda i: img.size[i])
    
    #scale both axis
    scale = dim/float(img.size[maxsize])
    
    return img.resize((int(img.size[0] * scale), int(img.size[1]*scale)))
    

    
def load_image(ids,dim=64):

    filepath = './data/images/'
    
    IM = np.empty((len(ids),dim,dim,1))
    
    for i, ids in enumerate(ids):
    
        images = rebuild_image(load_img(filepath + str(ids) + '.jpg',
grayscale=True),dim = dim)
        
        images = img_to_array(images)
        x = images.shape[0]
        y = images.shape[1]
        
        #centering
        l_bound_0 = int((dim - x) / 2)
        u_bound_0 = l_bound_0 + x
        l_bound_1 = int((dim - y) / 2)
        u_bound_1 = l_bound_1 + y
        
        IM[i,l_bound_0:u_bound_0,l_bound_1:u_bound_1,0:1] = images
        
        #scale it to grey
    return np.around(IM/ 255.0) 

* splitting the data with StratifiedShuffleSplit

In [15]:

from sklearn.model_selection import StratifiedShuffleSplit

random_state = 8
split = 0.8





def load_train_data(filepath, split = split, random_state = random_state):

    id, features_train, target_train = load_train(filepath)
    #image_train = load_image(id)
    
    sss = StratifiedShuffleSplit(n_splits = 1, train_size=split,random_state=random_state)
    train_i,val_i = next(sss.split(features_train,target_train))
    #,val_img   #,image_train[val_i]
    features_val,val_target = features_train[val_i], target_train[val_i]
    #,train_img  #,image_train[train_i]
    features_train,train_target = features_train[train_i],target_train[train_i]
    
    #,train_img   #,val_img
    return (features_train,train_target),(features_val,val_target)
    

def load_test_data(filepath):

    id, test_features = load_test(filepath)
    #test_images = load_image(id)
    
    return (id,test_features)#,test_images)
    
    

In [16]:
train,validation = load_train_data("./data/train.csv")

In [17]:
test = load_test_data("./data/test.csv")

In [46]:
type(test)

tuple

In [22]:
train_labels = train[1][:]
valid_labels = validation[1][:]

In [24]:
train[1][:].shape

(792,)

In [25]:
validation[1][:].shape

(99,)

## reformat function

In [27]:
def reformat(labels):
    labels = (np.arange(99) == labels[:,None]).astype(np.float32)
    return labels
train_labels = reformat(train_labels)
valid_labels = reformat(valid_labels)

In [28]:
input_layer = Input(shape=(64,64,1))

## buiding the pipe

In [31]:
#kernel_size: specifying the width and height of the 2D convolution window.
        

pipe_line = Convolution2D(8,5,5, border_mode = "same", W_regularizer='l1')(input_layer)
pipe_line =(Activation("relu"))(pipe_line)
pipe_line = (MaxPooling2D(pool_size=(2,2), strides = (2,2)))(pipe_line)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(8, (5, 5), padding="same", kernel_regularizer="l1")`
  after removing the cwd from sys.path.


In [ ]:
Conv2D()

## flatten the pipe

In [32]:
#flatten
pipe_line = Flatten()(pipe_line)


## fully connected layer (using dense class)

In [33]:
# fully connected layer
pipe_line = Dense(100, activation="relu")(pipe_line)
pipe_line = Dropout(.5)(pipe_line)


## output layer

In [34]:
#output layer
output_layer = Dense(99, activation="softmax")(pipe_line)

In [35]:
#lr ==> learning rate
sgd = SGD(lr= 0.008, momentum= 0.9)

In [38]:
pipe_line.compile(optimizer = "Adam", loss = "categorical_crossentropy",
             metrics = ["accuracy"]) ## Log_Loss for metrics???

AttributeError: 'Tensor' object has no attribute 'compile'

In [36]:
model_2layer = model.fit(train[1][:], train_labels,
                   validation_data = (validation[1][:],
                                     valid_labels),  nb_epoch = 1)

NameError: name 'model' is not defined